# MFSU vs ΛCDM: CMB Low-ℓ Analysis
This notebook fits the MFSU model to Planck 2018 TT data for low multipoles (ℓ ≤ 30), comparing it to ΛCDM.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample


In [ ]:

# Cargar CSV con datos filtrados de Planck TT
data = pd.read_csv("Planck_TT_low_ell.csv")
ell = data["ell"].values
cl_obs = data["Cl"].values
cl_err = data["Cl_err"].values


In [ ]:

def cl_mfsu(ell, A, d_f):
    return A * ell ** -(d_f - 1)

def cost(params, ell, cl_obs):
    A, d_f = params
    cl_pred = cl_mfsu(ell, A, d_f)
    return mean_squared_error(cl_obs, cl_pred, squared=False)


In [ ]:

result = minimize(cost, x0=[1e-10, 1.5], args=(ell, cl_obs), bounds=[(1e-12, 1e-8), (1.2, 2.0)])
A_best, d_f_best = result.x
print(f"A = {A_best:.2e}, d_f = {d_f_best:.4f}")


In [ ]:

d_f_boots = []
for _ in range(500):
    idx = np.random.choice(len(ell), len(ell), replace=True)
    ell_b, cl_b = ell[idx], cl_obs[idx]
    res = minimize(cost, x0=[A_best, d_f_best], args=(ell_b, cl_b))
    d_f_boots.append(res.x[1])
d_f_err = np.std(d_f_boots) * 1.96
print(f"95% CI for d_f: ±{d_f_err:.4f}")


In [ ]:

cl_pred = cl_mfsu(ell, A_best, d_f_best)
cl_lcdm = cl_mfsu(ell.astype(float), A_best, 2.0)
rmse_mfsu = mean_squared_error(cl_obs, cl_pred, squared=False)
rmse_lcdm = mean_squared_error(cl_obs, cl_lcdm, squared=False)
improvement = (rmse_lcdm - rmse_mfsu) / rmse_lcdm * 100

print(f"RMSE MFSU: {rmse_mfsu:.4f}")
print(f"RMSE ΛCDM: {rmse_lcdm:.4f}")
print(f"Improvement: {improvement:.2f}%")


In [ ]:

plt.figure(figsize=(10, 6))
plt.errorbar(ell, cl_obs, yerr=cl_err, fmt='ko', label='Planck 2018 (ℓ ≤ 30)')
plt.plot(ell, cl_pred, 'r-', label=f'MFSU (d_f = {d_f_best:.2f})')
plt.plot(ell, cl_lcdm, 'b--', label='ΛCDM (d_f = 2)')
plt.yscale('log')
plt.xlabel('Multipole moment ℓ')
plt.ylabel('C_ℓ [μK²]')
plt.title('MFSU vs ΛCDM Fit to Planck TT (ℓ ≤ 30)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("mfsu_vs_lcdm_fit.png")
plt.show()
